## Import Packages

In [1]:
using JuMP;
using Plots;
using HiGHS;
using CSV;
using DataFrames;
using MarketData;

## Load Data

In [2]:
function load_data(stocks::Array{String}, years::Int64)
    output = Array{Float64}[]
    t = Dates.now()
    period = YahooOpt(period1 = t - Year(years), period2 = t)
    for stock in stocks
        data = yahoo(stock, period)
        value = (values(data["High"]) + values(data["Low"]))/2;
        push!(output, value)
    end
    return hcat(output...)
end

function evaluate_factor(data::Matrix{Float64})
    m, n = size(data) 
    factor = copy(data)
    mean = zeros(n)
    for i in 1:n
        mean[i] = sum(data[:,i])/(1.05*m)
    end

    for i in 1:m
        factor[i,:] = (data[i,:] - mean) ./ mean
    end
    return factor
end

data = load_data(["PETR4.SA", "VALE3.SA", "AZUL4.SA", "BPAC11.SA"], 1);
factor = evaluate_factor(data);

## Model

In [3]:
function portifolio_simple(factor::Matrix{Float64}; fixed::Float64 = 0.0, β::Float64 = 0.0, α::Float64 = 0.05, BigM::Float64 = 1e3)

    m, n = size(factor) 
    p = 1/m
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    
    @variable(model, x[1:n] >= 0.0)
    @variable(model, y[1:m], Bin)
    
    @constraint(model, sum(x) <= 1.0)
    @constraint(model,[i = 1:m, j = 1:n], factor[i,j]*x[j] >= β - BigM*y[i])
    @constraint(model, sum(y*p) <= α)

    @objective(model, Max, sum(factor[i,j]*p*x[j] for i = 1:m, j = 1:n) + fixed*(1-sum(x)))

    optimize!(model)
    
    return model
end

portifolio_simple (generic function with 1 method)

In [4]:
function portifolio_complex(factor::Matrix{Float64}; fixed::Float64 = 0.0, β::Float64 = 0.0, α::Float64 = 0.05, BigM::Float64 = 1e3)

    m, n = size(factor) 
    p = 1/(m*n)
    
    model = Model(HiGHS.Optimizer)
    MOI.set(model, JuMP.MOI.Silent(), true)
    
    @variable(model, x[1:n] >= 0.0)
    @variable(model, y[1:m,1:n], Bin)
    
    @constraint(model, sum(x) <= 1.0)
    @constraint(model,[i = 1:m, j = 1:n], factor[i,j]*x[j] >= β - BigM*y[i,j])
    @constraint(model, sum(y*p) <= α)

    @objective(model, Max, sum(factor[i,j]*p*x[j] for i = 1:m, j = 1:n) + fixed*(1-sum(x)))

    optimize!(model)
    
    return model
end

portifolio_complex (generic function with 1 method)

In [10]:
model = portifolio_simple(factor, β = -0.1, α = 0.05)
@show value.(model[:x])
@show sum(value.(model[:x]))

value.(model[:x]) = [0.0, 0.28002975820133935, 0.19468194302265174, 0.5252882987760089]
sum(value.(model[:x])) = 1.0


1.0